In [12]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
import cv2
import matplotlib.pyplot as plt

In [3]:
# Parameters from Rulr
normalizePercentile = 1e-5
differencethreshold = 32
distanceThreshold = 10
minMeanPixelValueOnLine = 10

In [4]:
# Load data
with open('../../Application/bin/data/Patch.json') as file:
	patch = json.load(file)


In [5]:
# Get functions from next door
def find_node_by_name(node_name):
	for node_id in patch['Nodes']:
		if patch['Nodes'][node_id]['Name'] == node_name:
			return patch['Nodes'][node_id]
	raise(Exception("Node not found"))

def get_laser_capture(camera_capture, laserAddress):
	for laser_capture in camera_capture['laserCaptures']['captures']:
		if int(laser_capture['laserAddress']) == laserAddress:
			return laser_capture

calibrate_node_content = find_node_by_name('Calibrate')['Content']
camera_captures = calibrate_node_content['cameraCaptures']['captures']

In [6]:
def find_camera_capture(time, only_selected = False):
	for camera_capture in camera_captures:
		if only_selected and not camera_capture['Selected']:
			continue
		if camera_capture['time'] == time:
			return camera_capture
	raise(Exception("Couldn't find camera capture {0}".format(time)))

def find_laser_capture(camera_capture, laser_address, only_selected = False):
	laser_captures = camera_capture['laserCaptures']['captures']
	for laser_capture in laser_captures:
		if only_selected and not laser_capture['Selected']:
			continue
		if laser_capture['laserAddress'] == laser_address:
			return laser_capture
	raise(Exception("Couldn't find laser_capture {0}".format(laser_address)))

In [7]:
time = '06:25'
laser_address = 35
laser_capture = find_laser_capture(find_camera_capture(time), laser_address)

In [8]:
def get_beam_capture_images(laser_capture, only_selected = False):
	on_images = []
	off_images = []
	index = 0
	for beam_capture in laser_capture['beamCaptures']['captures']:
		if only_selected and not beam_capture['Selected']:
			continue
		print("Loading {0}".format(index))
		on_image = cv2.imread(beam_capture['onImage']['localCopy'], cv2.IMREAD_GRAYSCALE)
		off_image = cv2.imread(beam_capture['offImage']['localCopy'], cv2.IMREAD_GRAYSCALE)
		on_images.append(on_image)
		off_images.append(off_image)
		index = index + 1
	return on_images, off_images

In [9]:
on_images, off_images = get_beam_capture_images(laser_capture)

Loading 0
Loading 1
Loading 2
Loading 3
Loading 4
Loading 5
Loading 6
Loading 7
Loading 8


In [10]:
difference = cv2.subtract(on_images[0], off_images[0])


In [43]:
fig = px.imshow(difference, title="{0} #{1}".format(time, laser_address))
fig.show()

In [ ]:
plt.imshow(difference)
plt.show()

In [13]:
def get_difference_method_2(on_image, off_image):
	kernel = np.ones((5, 5), np.uint8)
	background = cv2.dilate(off_image, kernel, iterations=2)
	difference = cv2.subtract(on_image, background)
	return difference

In [ ]:
difference_2 = get_difference_method_2(on_images[0], off_images[0])

In [14]:
fig = px.imshow(difference_2, title="{0} #{1} (method 2)".format(time, laser_address))
fig.show()